In [2]:
import tensorflow as tf

import os
import pathlib
import time
import datetime

import numpy as np

import gdown

from matplotlib import pyplot as plt
from IPython import display

import numpy as np
from sklearn.model_selection import GridSearchCV

import sys

sys.path.append('/home/natanael/deblurring_images')

from utils import load, load_image_train, load_image_test
from models.pixpix import Pix2Pix

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


2023-07-21 13:36:33.843044: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [7]:
path = "littleGoProBase"
BUFFER_SIZE = 400
BATCH_SIZE = 1

IMG_WIDTH = 256
IMG_HEIGHT = 256
OUTPUT_CHANNELS = 3

In [8]:
tam_train = len( list(tf.data.Dataset.list_files(f'{path}/train/*.png' )) )
tam_test = len( list(tf.data.Dataset.list_files(f'{path}/test/*.png')) )

print(f"Train tam: {tam_train}")
print(f"Teste tam: {tam_test}")

2023-07-21 13:37:34.552802: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-21 13:37:34.559579: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-21 13:37:34.589958: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-21 13:37:34.590229: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.78GiB deviceMemoryBandwidth: 245.91GiB/s
2023-07-21 13:37:34.590252: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-07-21 13:37:34.608194: I tensorflow/stream_executor/platform/defa

Train tam: 10000
Teste tam: 5000


In [9]:
train_dataset = tf.data.Dataset.list_files(f'{path}/train/*.png')
train_dataset = train_dataset.map(load_image_train, num_parallel_calls=tf.data.AUTOTUNE).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [10]:
test_dataset = tf.data.Dataset.list_files(f'{path}/test/*.png')
test_dataset = test_dataset.map(load_image_test).batch(BATCH_SIZE)

In [11]:
LAMBDA = 100
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
mse_loss = tf.keras.losses.MeanSquaredError()

In [12]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # Mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
  
  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss

In [13]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [14]:
!mkdir ./train_checkpoints_mse_loss

mkdir: cannot create directory ‘./train_checkpoints_mse_loss’: File exists


In [15]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15, 15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [16]:
!mkdir ./logs_mse_loss

mkdir: cannot create directory ‘./logs_mse_loss’: File exists


In [17]:
log_dir="./logs_mse_loss/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [18]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
checkpoint_dir = './train_checkpoints_mse_loss'

pix = Pix2Pix()

pix.compile(
  otimazer_generator=generator_optimizer,
  otimazer_discriminator=discriminator_optimizer, 
  loss_generator=generator_loss, 
  loss_discriminator=discriminator_loss, 
  checkpoint_dir=checkpoint_dir
)

In [19]:
np.arange(100, 1000, 100)

array([100, 200, 300, 400, 500, 600, 700, 800, 900])

In [ ]:


# Define the hyperparameters to search
param_grid = {
    'learning_rate': [0.001  ],
    'max_depth': [3],
    'n_estimators': [100],
}

train_dataset_ = []
test_dataset_ = []
for batch in train_dataset:
  train_dataset_.append(batch)

for bt in test_dataset:
    test_dataset_.append(bt)

train_dataset_ = train_dataset_[:400]
test_dataset_ = test_dataset_[:400]
grid_search = GridSearchCV(
    estimator=pix,
    param_grid=param_grid,
    cv=5,
    verbose=2,
)

with tf.device('/GPU:0'):
    grid_search.fit(train_dataset_, test_dataset_)

# Print the best parameters
print(grid_search.best_params_)


2023-07-21 13:37:39.640700: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-07-21 13:37:39.642344: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2599990000 Hz
